#### Creating Tokens 

In [1]:
with open("the-verdict.txt", "r", encoding="utf-8") as f : 
    raw_text = f.read() 
print("Total number of character : ", len(raw_text))
print(raw_text[:99])

Total number of character :  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


#####  Our goal is to tokenize this 20479 characters short story into individual words and special characters that we can then turn into embeddings for llm training 

In [2]:
import re 
text = "Hello, world. this, is a test" 
res = re.split(r'(\s)', text )
print(res)

['Hello,', ' ', 'world.', ' ', 'this,', ' ', 'is', ' ', 'a', ' ', 'test']


In [3]:
## we also need , . to be spereate 
res = re.split(r'([,.]|\s)', text)
print(res)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'this', ',', '', ' ', 'is', ' ', 'a', ' ', 'test']


In [4]:
### removeing the white space characters 
res = [item for item in res if item.strip()]
print(res)

['Hello', ',', 'world', '.', 'this', ',', 'is', 'a', 'test']


In [5]:
test = "Hello, world. Is this -- a test ?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', test)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


##### Now lets apply in our main dataset raw_text

In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
len(preprocessed)

4690

#### Creating Token id's 

In [8]:
all_words = sorted(set(preprocessed)) ## unique and sorted
vocab_size = len(all_words)  # only unique words 
print(vocab_size)

1130


In [9]:
vocab = {token : integer for integer,token in enumerate(all_words)}

In [10]:
for i , item in enumerate(vocab.items()): 
    print(item)
    if i >= 50: 
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


#### Lets implement a Tokeninzer Class in python  
* Encode method :- sample text -> tokenized text -> token ids 
* Decode method :- token ids -> tokenized test -> sample text 

In [11]:
class SimpleTokenizerV1: 
    def __init__(self, vocab): 
        self.str_to_int = vocab 
        self.int_to_str = {i:s for s,i in vocab.items()}
    def encode(self, text): 
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids 
    def decode(self, ids): 
        text = " ".join([self.int_to_str[i] for i in ids]) 
        # replace spaces before the specifed punctuation 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
        Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text) 
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(ids) 

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [14]:
### there is one promblem that if the text is not present int the vocab then it will throw error to us 

#### Special Context Tokens  : 
* In this section , we will modify the tokenizer to handle unknow words
* In particular , we will modify the vocab and tokenizer we implemented in the previous section , tokenizerv2 to support two new tokens, <|unk|> and <|endoftext|>

In [15]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token : integer for integer,token in enumerate(all_tokens)}

In [16]:
len(vocab.items())

1132

In [17]:
for i , item in enumerate(list(vocab.items()) [-5:]): 
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [18]:
class SimpleTokenizerV2:   ## V2 
    def __init__(self, vocab): 
        self.str_to_int = vocab 
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self, text): 
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
                item if item in self.str_to_int
                else "<|unk|>"
                for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids 
    def decode(self, ids): 
        text = " ".join([self.int_to_str[i] for i in ids]) 
        # replace spaces before the specifed punctuation 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [19]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello , do you like tea ?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello , do you like tea ? <|endoftext|> In the sunlit terraces of the palace.


In [20]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [21]:
### above 1131 is an unk text and in 1130 is the endoftext

In [22]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

#### BYTE PAIR ENCODING 
* This Section covers a more sophisticated tokenization scheme based on a concpet called byte pair encoding (BPE).
* The BPE tokenizer covered in this section is used to train LLMs such as GPT-3 , GPT-2 and the orginal models used in ChatGpt.

##### Since implementing BPE can be relatively 

In [27]:
tokenizer = tiktoken.get_encoding("gpt2")
tokenizer.n_vocab

50257

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces "
    "of someunkownPlace."
)

integers = tokenizer.encode(
    text,
    allowed_special={"<|endoftext|>"}
)
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 2954, 593, 27271, 13]


In [29]:
Strings = tokenizer.decode(integers)
print(Strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunkownPlace.


#### CREATING INPUT-TARGET PAIR

##### In this section we implement a data loader that fetches the input-target pair using a sliding window apporach. 

In [5]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

with open("the-verdict.txt", "r", encoding="utf-8") as f : 
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [31]:
# Creating an a sample_text 
enc_sample = enc_text[50:]

##### One of the easiest and most intuitive ways to create the input-target pairs for the nextword prediction task 
##### is to create x as input tokens and y as a targets , which input shifts by 1 . 

and the context size determines how many tokens are included in the input 

In [32]:
context_size = 4 # length of the input 
# the context_size of 4 means that the model is trained to look at a sequence of 4 words (or tokens)
# to predict the next word in the sequence . 
# the input x is the first 4 tokens [1,2,3,4] and the target y is the next 4 tokens [2,3,4,5]

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x : {x}")
print(f"y : {y}")

x : [290, 4920, 2241, 287]
y : [4920, 2241, 287, 257]


In [33]:
for i in range(1, context_size+1): 
    context = enc_sample[:i] 
    desired = enc_sample[i]
    print(context, "----->", desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [34]:
for i in range(1, context_size+1): 
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


##### There's only one more task before we can turn the tokens into embeddings : implementing an efficient data loader that iterates 
##### over the input dataset and returns the inputs and targets as PyTorch tensors, which can be thought as multidimensional arrays . 

* In particularly, we are interested in returning two tensors . an input tensor containing the text that the LLM sees and a target tensor that includes
  the targets for the LLM to predict. 

#### IMPLEMENTING AN A DATA LOADER 
* 1. Tokenize the entire text
  2. Use a sliding window to chunk the book into overlapping sequence of max_length
  3. Return the total number of rows in the dataset
  4. Return a single row from the dataset 

In [6]:
from torch.utils.data import Dataset, DataLoader 

class GPTDatasetV1(Dataset): 

    def __init__(self, txt, tokenizer, max_length, stride): 
        
        self.input_ids = []
        self.target_ids = [] 
        
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids)- max_length, stride): 
            
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self): 
        return len(self.input_ids)

    def __getitem__(self, idx): 
        return self.input_ids[idx], self.target_ids[idx] 
    

##### The following code will use the GPTdatasetV1 to load the inputs in batches via a pytorch DataLoader: 

##### 
* 1. Initialize the tokenizer
  2. Create dataset
  3. drop_last = true drops the last batch if it is shorter than the specified batch_size to prevent losss spikes during training
  4. the number of CPU processes to use for preprocessing 

In [7]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride = 128,
                         shuffle=True, drop_last = True,
                         num_workers=0): 
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last, 
        num_workers=num_workers 
    )
    return dataloader

In [8]:
with open("the-verdict.txt", "r", encoding="utf-8") as f : 
    raw_text = f.read() 

In [9]:
import torch 
print("Pytorch version:", torch.__version__)

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)

print(first_batch)

Pytorch version: 2.9.1+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [39]:
####  firt_batch have two tensors input and the target 
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [39]:
### exprement batch size 8 
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n" , inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


#### CREATING TOKEN EMBEDDINGS

In [41]:
### example 
input_ids = torch.tensor([2,3,5,1])

In [42]:
### lets assume for now 6 words are there and vector dim is 3 
vocab_size = 6 
vector_dim = 3 

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, vector_dim)  ## just an lookup embedding layer

In [43]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [46]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [48]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


#### POSITIONAL EMBEDDING

In [11]:
vocab_size = 50257
ouput_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size,ouput_dim)


In [12]:
max_length = 4 
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, 
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, target = next(data_iter)

In [17]:
print("Inputs:\n" , inputs)
print("\nShape of the input:\n", inputs.shape)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Shape of the input:
 torch.Size([8, 4])


##### Now lets just use the embedding layer to convert this inputs ids into 256 dim vectors 

In [18]:
token_embedding = token_embedding_layer(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


##### For a GPT models absolute embedding approch , we just need to create another embedding layer that as 
the dim as the token 

In [19]:
context_length = max_length 
poss_embedding_layer = torch.nn.Embedding(context_length, ouput_dim)

In [20]:
poss_embedding = poss_embedding_layer(torch.arange(max_length))
print(poss_embedding.shape)

torch.Size([4, 256])


In [21]:
input_embedding = token_embedding + poss_embedding 

In [23]:
print(input_embedding[0][1])

tensor([ 1.4586, -2.7059, -1.6009,  2.0733, -1.0641,  0.5459,  0.0826,  0.6918,
         1.6816, -2.1791,  3.5439, -2.1453, -0.6077,  1.5427,  2.1817,  1.9739,
         0.7902,  0.2881, -2.0708,  0.7417,  2.0146,  0.9197, -0.7231,  0.0834,
         0.5060, -0.3768,  1.1812, -0.8178, -1.6132,  0.4905,  0.5419,  0.1770,
        -0.6924,  1.2096, -0.3171, -1.3824,  0.2527, -0.9191,  0.0518, -0.3535,
         0.8389,  1.3452, -0.2666,  1.8771, -1.9147, -0.9975, -2.1101,  2.2615,
        -1.1338, -2.1860,  0.5529,  1.7545,  0.0125, -2.5067,  0.1044, -1.4572,
        -0.8126,  0.0899,  0.1762,  0.0065, -1.1046, -2.7164,  0.0343,  1.7826,
         0.2154,  0.4759,  2.1575,  1.4909,  1.8447, -2.4236, -1.9045, -1.1304,
         1.9577, -1.5996, -0.6046,  1.0552, -0.3769,  1.2873,  1.6838,  0.1219,
        -1.6466, -1.4231, -1.2849, -1.0223,  1.5223, -1.9278, -2.1009, -0.3794,
        -0.6117,  0.4650, -0.0405,  1.2099,  2.3863,  1.8101, -0.0613,  0.9611,
         0.2628, -0.7236,  0.3854,  0.45

##### 
    token_embedding[0][1]   # meaning of the word
    + pos_embedding[1]      # position = 1 (second word)
